In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.linalg import eigvals

In [7]:
# checking if a matrix is p-matrix
import numpy as np
from itertools import combinations
import pandas as pd

def generate_subsets(n):
    # Generate all possible subsets of size 1 to n.
    for size in range(1, n + 1):
        for subset in combinations(range(n), size):
            yield subset
            # print(subset)

def check_p(matrix):
    n = len(matrix)

    # Compute the determinant of the whole matrix first.
    determinant = np.linalg.det(matrix)
    if determinant <= 0:
        return "Not a p-matrix"  # Return immediately if the whole matrix has a non-positive determinant

    # Check the determinants of diagonal elements (smallest subset) next.
    for i in range(n):
        submatrix = matrix[i, i]
        if submatrix <= 0:
            return "Not a p-matrix"  # Return immediately if any diagonal element is non-positive

    # Initialize the generator to get subsets.
    subset_generator = generate_subsets(n)

    # Check the determinants of other subsets.
    for subset in subset_generator:
        if len(subset) > 1:
            submatrix = matrix[np.ix_(subset, subset)]
            determinant = np.linalg.det(submatrix)
            if determinant <= 0:
                return "Not a p-matrix"  # Return immediately if a non-positive determinant is found

    return "All principal minors are positive -> p-matrix"


In [8]:
import numpy as np

def generate_non_symmetric_matrix(n):
    # Generate a random matrix with values between -1 and 1
    matrix = np.random.uniform(-1, 1, (n, n))

    # Set diagonal elements to 1
    np.fill_diagonal(matrix, 1)

    # Check sub-matrix determinants and adjust as needed
    for i in range(n):
        for j in range(i + 1, n):
            sub_matrix = matrix[:j+1, :j+1]  # Extract sub-matrix
            while np.linalg.det(sub_matrix) < 0:
                # Find a random element in the sub-matrix
                row_idx, col_idx = np.random.randint(0, j+1), np.random.randint(0, j+1)
                # Flip the sign of the element
                matrix[row_idx, col_idx] *= -1
    
    return matrix

# Define the size of the matrix (change n to your desired value)
n = 4
result_matrix = generate_non_symmetric_matrix(n)
print(result_matrix)

[[ 1.          0.35535211  0.54990837  0.18605189]
 [-0.19747944  1.          0.80191627 -0.81793347]
 [-0.99081524 -0.89966169  1.         -0.98655034]
 [-0.79146721  0.87756217 -0.66432958  1.        ]]


In [20]:
def schur_complement(matrix, block_indices):
    A = matrix[block_indices[0]:block_indices[1], block_indices[0]:block_indices[1]]
    B = matrix[block_indices[0]:block_indices[1], block_indices[1]:]
    C = matrix[block_indices[1]:, block_indices[0]:block_indices[1]]
    D = matrix[block_indices[1]:, block_indices[1]:]

    inv_D = np.linalg.inv(D)
    S = A - B @ inv_D @ C

    return S

# Create a random 4x4 matrix
matrix = generate_non_symmetric_matrix(4)

# Define the block indices for partitioning (e.g., block at row 2 and column 2)
block_indices = (2, 4)

# Compute the Schur complement
schur_matrix = schur_complement(matrix, block_indices)

print("Original Matrix:")
print(matrix)

print("\nSchur Complement:")
print(schur_matrix)

Original Matrix:
[[ 1.          0.17275446  0.00698661  0.62637853]
 [-0.884951    1.          0.06665338  0.33791885]
 [-0.17629458  0.22182929  1.          0.16269795]
 [-0.69268878  0.88167653  0.56353493  1.        ]]

Schur Complement:
[[1.         0.16269795]
 [0.56353493 1.        ]]


In [21]:
check_p(result_matrix)

'All principal minors are positive -> p-matrix'

In [22]:
block_indices = (2, 4)
schur_comp = schur_complement(result_matrix, block_indices)
check_p(schur_comp)

'All principal minors are positive -> p-matrix'

In [25]:
def find_p_matrix_with_non_p_schur(n, max_attempts=100000):
    for _ in range(max_attempts):
        # Generate a random n x n matrix with positive elements
        matrix = generate_non_symmetric_matrix(n)

        # Check if the generated matrix is a P-matrix
        if not check_p(matrix):
            print("Generated matrix is not a P-matrix.")
            return matrix

        # Define the block indices for partitioning
        block_indices = (n // 2, n)

        # Compute the Schur complement
        schur_matrix = schur_complement(matrix, block_indices)

        # Check if the Schur complement is a P-matrix
        if not check_p(schur_matrix):
            print("Generated P-matrix has a non-P-matrix Schur complement.")
            print("Generated P-matrix:")
            print(matrix)
            print("\nSchur Complement:")
            print(schur_matrix)
            return matrix

    print("No P-matrix with a non-P-matrix Schur complement found after", max_attempts, "attempts.")
    return None

# Example usage:
n = 20  # Adjust the size of the matrix as needed
p_matrix = find_p_matrix_with_non_p_schur(n)
if p_matrix is not None:
    print("Found a P-matrix where its Schur complement is not a P-matrix.")

No P-matrix with a non-P-matrix Schur complement found after 100000 attempts.
